<a href="https://colab.research.google.com/github/Friedrichz/crypto_tooling/blob/main/track_vc_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install messari

In [117]:
from messari.messari import Messari
import pandas as pd
import numpy as np

messari_api_key = "88bbfa31-1b0c-442b-b748-cc13ed2cab1a"
messari = Messari(messari_api_key)

In [4]:
# Fetch raw data
li_dic = [] 
for i in range(1, 5, 1):
    data = messari.get_all_assets(page=i, limit=500, to_dataframe=True)
    li_dic.append(data)

# Merge raw API data into one list with records 
data = [j for i in li_dic for j in i["data"]]
len(data)

# Inspect coins
# [(n,i) for n, i in enumerate([i["name"] for i in data])]

# Cleanup records 

In [108]:
def get_nested_items(list_records, attr): 
  if list_records is None:
    return []
  else: 
    return [i[attr] for i in list_records]

def clean_sales_round_title(title): 
  if title is None: pass
  else:
    # Remove digits
    title = ''.join([i for i in title if not i.isdigit()]) 
    # Lowercase & strip trailing/leading spaces
    title = title.lower().strip()
    return title

def clean_sales_rounds(list_records): 
  if list_records is None:
    pass
  else:
    [i.update(title_clean=clean_sales_round_title(i["title"])) for i in list_records]
    return

# Get list of sales_rounds labels to identify venture round / seed sale 
sales_rounds_titles = set(j for i in data for j in get_nested_items(i["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"], "title"))

# Taken manually from list of titles in data 
private_sales_rounds_titles = [
  'Fundraising ', 
  'Pre Seed Round', 
  'Pre-Sale SAFT', 
  'Private Presale', 
  'Private Sale', 
  'Private Sale1 ',
  'SAFT',
  'Seed', 
  'Seed ', 
  'Seed Funding', 
  'Seed Round', 
  'Seed Round (SAFT) ', 
  'Seed Sale', 
  'Seed Round (Crypto)', 
  'Seed Token Sale',
  'Strategic Funding Round',
  'Strategic Investments',
  'Strategic Private Sale',
  'Strategic Round',
  'Strategic Sale',
  'Token Pre-Sale',
  'Token Private Round',
  'Token Seed Round',
  'Venture Capital',
  'Venture Funding',
  'Venture Round',
]

# Clean up sales round titles & add "title_clean" to records
private_sales_rounds_titles_clean = set(clean_sales_round_title(i) for i in private_sales_rounds_titles)
print(len(private_sales_rounds_titles_clean)) # Include 24 titles that point to private token sales
_ = [clean_sales_rounds(j["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]) for j in data]

24


In [111]:
# Extract (current) market data --> metrics: token_sale_stats>sale_proceeds_usd, supply_distribution
def get_market_data(record): 
  # Get current supply (see docs: https://messari.io/article/messari-proprietary-methods)
  circ_supply = record["metrics"]["supply"]["circulating"] # liquid supply that excludes project, foundation or founder units which have not been yet sold
  liquid_supply = record["metrics"]["supply"]["liquid"] # supply taht is visible on-chain and which is not known to have any programmatic or contractual restrictions

  # Get ath price data
  ath_price = record["metrics"]["all_time_high"]["price"]
  ath_date = record["metrics"]["all_time_high"]["at"]

  # Get cycle low price data
  cl_price = record["metrics"]["cycle_low"]["price"]
  cl_date = record["metrics"]["cycle_low"]["at"]

  # Get current mcap
  mcap = record["metrics"]["marketcap"]["current_marketcap_usd"]
  liquid_mcap = record["metrics"]["marketcap"]["liquid_marketcap_usd"]

  # Get current price data
  current_price = record["metrics"]["market_data"]["price_usd"]

  # Get ROI data
  pct_changes_keys = ["percent_change_last_1_year", "percent_change_year_to_date", "percent_change_last_3_months", "percent_change_last_1_month", "percent_change_month_to_date", "percent_change_last_1_week",]
  pct_changes = {k:v for k,v in record["metrics"]["roi_data"].items() if k in pct_changes_keys}

  return dict(
      current_price = current_price,
      circ_supply = circ_supply, 
      liquid_supply = liquid_supply, 
      ath_price = ath_price, 
      ath_date = ath_date, 
      cl_price = cl_price, 
      cl_date = cl_date, 
      mcap = mcap, 
      liquid_mcap = liquid_mcap,
      **pct_changes       
  )


In [138]:
# Extract initial private fundraising data -->  profile: investors>organizations, economics>launch>fundraising
def get_fundraise_data(record, private_sales_rounds_titles_clean=private_sales_rounds_titles_clean):
  # Init
  investors = []
  first_private_sale_date = np.nan
  last_private_sale_date = np.nan
  num_tokens_private_sale = np.nan
  tot_usd_raised_private_sale = np.nan
  min_price_private_sale = np.nan
  max_price_private_sale = np.nan
  wavg_price_private_sale = np.nan 
  init_supply = np.nan
  fdv_private_sale = np.nan
  asset_collected = np.nan

  coin_investors = record["profile"]["investors"]["organizations"] 
  if coin_investors != None:
    # Get list of private investors
    investors = [i["name"] for i in coin_investors]

  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds != None:
    # Filter records of private token sales
    private_rounds = [i for i in sales_rounds if i["title_clean"] in private_sales_rounds_titles_clean]
    if private_rounds:
      # Get earliest & last date of private sales
      first_private_sale_date = min(filter(None, [i["end_date"] for i in private_rounds]), default=np.nan)
      last_private_sale_date = max(filter(None, [i["end_date"] for i in private_rounds]), default=np.nan)

      # Calc total number of tokens sold during private sales
      num_tokens_private_sale = sum(filter(None, [i["native_tokens_allocated"] for i in private_rounds]))

      # Calc total USD amount raised in private sales
      tot_usd_raised_private_sale = sum(filter(None, [i["amount_collected_in_usd"] for i in private_rounds]))

      # Get last private sale token price 
      min_price_private_sale = min(filter(None, [i["equivalent_price_per_token_in_usd"] for i in private_rounds]), default=np.nan)
      max_price_private_sale = max(filter(None, [i["equivalent_price_per_token_in_usd"] for i in private_rounds]), default=np.nan)

      # Calc weighted avg token price of private sales
      # wavg_price_private_sale = sum(filter(None, [i["amount_collected_in_usd"]*i["equivalent_price_per_token_in_usd"] for i in private_rounds])) / tot_usd_raised_private_sale

      # Get total token supply
      init_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

      # Calc FDV based on last private sale token price
      # fdv_private_sale = max_price_private_sale * init_supply

      # Safety check: currency collected in private fundraise
      asset_collected = set(i["asset_collected"] for i in private_rounds)

  return dict(
        investors = investors, 
        first_private_sale_date = first_private_sale_date, 
        last_private_sale_date = last_private_sale_date, 
        num_tokens_private_sale = num_tokens_private_sale, 
        tot_usd_raised_private_sale = tot_usd_raised_private_sale, 
        min_price_private_sale = min_price_private_sale, 
        max_price_private_sale = max_price_private_sale, 
        # wavg_price_private_sale = wavg_price_private_sale, 
        init_supply = init_supply, 
        # fdv_private_sale = fdv_private_sale,
        asset_collected = asset_collected,
    )

In [194]:
# Extract initial private fundraising data -->  profile: investors>organizations, economics>launch>fundraising
# Using pandas
def get_fundraise_data(record, private_sales_rounds_titles_clean=private_sales_rounds_titles_clean):
  # Init
  investors = []
  first_private_sale_date = np.nan
  last_private_sale_date = np.nan
  num_tokens_private_sale = np.nan
  tot_usd_raised_private_sale = np.nan
  min_price_private_sale = np.nan
  max_price_private_sale = np.nan
  wavg_price_private_sale = np.nan 
  init_supply = np.nan
  fdv_private_sale = np.nan
  asset_collected = np.nan

  # Get total token supply
  init_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

  # Get list of private investors
  coin_investors = record["profile"]["investors"]["organizations"] 
  if coin_investors != None:
    investors = [i["name"] for i in coin_investors]

  # Get details on sales rounds
  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds:
    df = pd.DataFrame.from_records(sales_rounds)

    # Filter records of private token sales
    df1 = df[df.title_clean.isin(private_sales_rounds_titles_clean)]
    # Get earliest & last date of private sales
    first_private_sale_date = df1.end_date.astype(str).min()
    last_private_sale_date = df1.end_date.astype(str).max()

    # Calc total number of tokens sold during private sales
    num_tokens_private_sale = df1.native_tokens_allocated.sum()
    # Calc total USD amount raised in private sales
    tot_usd_raised_private_sale = df1.amount_collected_in_usd.sum()

    # Get min/max private sale token price 
    min_price_private_sale = df1.equivalent_price_per_token_in_usd.min()
    max_price_private_sale = df1.equivalent_price_per_token_in_usd.max()

    # Calc weighted avg token price of private sales
    if num_tokens_private_sale:
      wavg_price_private_sale = sum(df1.equivalent_price_per_token_in_usd * df1.native_tokens_allocated) / num_tokens_private_sale

    # Calc FDV based on last private sale
    if init_supply:
      fdv_private_sale = max_price_private_sale * init_supply

    # Safety check: currency collected in private fundraise
    asset_collected = df1.asset_collected.unique().tolist()

  return dict(
        investors = investors, 
        first_private_sale_date = first_private_sale_date, 
        last_private_sale_date = last_private_sale_date, 
        num_tokens_private_sale = num_tokens_private_sale, 
        tot_usd_raised_private_sale = tot_usd_raised_private_sale, 
        min_price_private_sale = min_price_private_sale, 
        max_price_private_sale = max_price_private_sale, 
        wavg_price_private_sale = wavg_price_private_sale, 
        init_supply = init_supply, 
        fdv_private_sale = fdv_private_sale,
        asset_collected = asset_collected,
    )


In [195]:
# Get final data for all records
final_records = []

for record in data: 
  final_records.append(
      dict(
        symbol = record["symbol"], 
        name = record["name"], 
        link = "https://messari.io/asset/{}".format(record["slug"]),
        ** get_market_data(record),
        ** get_fundraise_data(record), 
      )
  )

In [214]:
# Create pd df
df = pd.DataFrame.from_records(final_records)
print(df.shape)

# Add ROI column since private sale
df["pct_change_private_sale"] = (df.current_price / df.wavg_price_private_sale - 1) * 100
df["pct_change_valuation"] = (df.mcap / df.fdv_private_sale - 1) * 100

df.head()

(2000, 29)


,symbol,name,link,current_price,circ_supply,liquid_supply,ath_price,ath_date,cl_price,cl_date,...,num_tokens_private_sale,tot_usd_raised_private_sale,min_price_private_sale,max_price_private_sale,wavg_price_private_sale,init_supply,fdv_private_sale,asset_collected,pct_change_private_sale,pct_change_valuation
0,BTC,Bitcoin,https://messari.io/asset/bitcoin,23208.090131,1.910791e+07,1.912012e+07,68721.934821,2021-11-10T14:00:00Z,17664.958903,2022-06-18T20:15:00Z,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN
1,ETH,Ethereum,https://messari.io/asset/ethereum,1679.456694,1.217644e+08,1.165134e+08,4847.573312,2021-11-10T15:30:00Z,897.060060,2022-06-18T20:30:00Z,...,0.0,0.0,NaN,NaN,NaN,7.200368e+07,NaN,[],NaN,NaN
2,USDT,Tether,https://messari.io/asset/tether,0.999446,6.605698e+10,NaN,1.261429,2016-02-12T08:30:00Z,0.000021,2022-07-02T12:30:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDC,USD Coin,https://messari.io/asset/usd-coin,1.000000,5.445763e+10,NaN,1.127539,2018-11-18T09:15:00Z,0.872572,2019-10-03T18:30:00Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BNB,BNB,https://messari.io/asset/binance-coin,284.816286,1.613373e+08,1.083456e+08,689.922433,2021-05-10T06:15:00Z,185.648220,2022-06-18T20:15:00Z,...,0.0,0.0,NaN,NaN,NaN,2.000000e+08,NaN,[],NaN,NaN


In [198]:
df.describe()

,current_price,circ_supply,liquid_supply,ath_price,cl_price,mcap,liquid_mcap,percent_change_last_1_week,percent_change_last_1_month,percent_change_last_3_months,percent_change_last_1_year,percent_change_month_to_date,percent_change_year_to_date,num_tokens_private_sale,tot_usd_raised_private_sale,min_price_private_sale,max_price_private_sale,wavg_price_private_sale,init_supply,fdv_private_sale
count,1.201000e+03,8.820000e+02,2.320000e+02,9.240000e+02,9.240000e+02,8.370000e+02,2.280000e+02,1475.000000,1436.000000,1.136000e+03,657.000000,1483.000000,1.483000e+03,3.400000e+02,3.400000e+02,66.000000,66.000000,43.000000,2.950000e+02,5.700000e+01
mean,2.079116e+02,8.576766e+13,4.484446e+10,8.932645e+13,5.366919e+11,1.285592e+09,3.895735e+09,10.366774,105.307010,6.056433e+03,-34.906724,-0.022796,-3.876117e+06,8.082743e+07,3.743098e+06,2.207732,2.512198,3.250286,3.398973e+12,2.261382e+09
std,2.022685e+03,1.778992e+15,5.275332e+11,2.715289e+15,1.631402e+13,1.723657e+10,3.214127e+10,46.309066,2623.517373,1.740818e+05,247.434192,1.615557,1.493370e+08,4.592670e+08,1.962088e+07,14.755978,15.446360,18.598226,5.822126e+13,1.654786e+10
min,7.726740e-17,0.000000e+00,1.085400e+04,5.087311e-12,3.339383e-18,0.000000e+00,3.428302e+04,-99.975477,-99.994543,-9.999988e+01,-99.999717,-49.961220,-5.750928e+09,0.000000e+00,0.000000e+00,0.000177,0.000177,0.000625,0.000000e+00,4.468749e+05
25%,3.382700e-03,1.121043e+07,7.551852e+07,1.865755e-01,3.549076e-03,6.345827e+05,2.940695e+07,0.000000,0.000000,-5.949375e+01,-83.238530,0.000000,-7.948641e+01,0.000000e+00,0.000000e+00,0.010000,0.016000,0.010217,3.121668e+07,7.900000e+06
50%,4.288998e-02,1.090607e+08,3.368675e+08,1.583410e+00,3.973713e-02,9.679915e+06,1.172420e+08,5.171377,11.066980,-4.371209e+01,-65.050772,0.000000,-6.151121e+01,0.000000e+00,0.000000e+00,0.035800,0.080000,0.058000,2.361990e+08,2.600000e+07
75%,5.156192e-01,7.526655e+08,1.000000e+09,1.341184e+01,4.755520e-01,8.313414e+07,6.149961e+08,14.823498,32.346315,-2.607121e+01,-39.949408,0.000000,-2.209658e+01,0.000000e+00,3.000000e+04,0.195000,0.240000,0.190000,1.000000e+09,1.080000e+08
max,2.488004e+04,4.019216e+16,7.959855e+12,8.253764e+16,4.959034e+14,4.434179e+11,4.437012e+11,989.588610,97906.925788,5.821298e+06,5414.050085,26.835412,2.688273e+06,4.707861e+09,2.850000e+08,120.000000,125.000000,122.031161,9.999916e+14,1.250000e+11


In [215]:
df[df.fdv_private_sale > 0]

,symbol,name,link,current_price,circ_supply,liquid_supply,ath_price,ath_date,cl_price,cl_date,...,num_tokens_private_sale,tot_usd_raised_private_sale,min_price_private_sale,max_price_private_sale,wavg_price_private_sale,init_supply,fdv_private_sale,asset_collected,pct_change_private_sale,pct_change_valuation
8,SOL,Solana,https://messari.io/asset/solana,42.417313,3.460267e+08,5.387110e+08,258.929066,2021-11-06T21:30:00Z,2.618111e+01,2022-06-14T01:45:00Z,...,8.846599e+07,5460000.0,0.040000,0.250000,0.061781,5.000000e+08,1.250000e+08,[USD],68557.768109,11635.942405
10,DOT,Polkadot,https://messari.io/asset/polkadot,8.657735,9.875793e+08,1.224132e+09,55.076698,2021-11-04T13:45:00Z,6.457865e+00,2022-06-13T11:15:00Z,...,8.420800e+05,102760000.0,120.000000,125.000000,122.031161,1.000000e+09,1.250000e+11,"[USD, BTC]",-92.905308,-93.147376
12,MATIC,Polygon,https://messari.io/asset/polygon,0.913680,8.034804e+09,9.178127e+09,2.904894,2021-12-27T03:30:00Z,3.227761e-01,2022-06-18T20:30:00Z,...,2.090000e+08,165110.0,0.000790,0.000790,0.000790,1.000000e+10,7.900000e+06,[None],115555.663238,92845.931193
13,AVAX,Avalanche,https://messari.io/asset/avalanche,23.708614,2.843227e+08,3.076503e+08,146.109798,2021-11-21T14:00:00Z,1.379976e+01,2022-06-19T05:30:00Z,...,1.800000e+07,5940000.0,0.330000,0.330000,0.330000,3.600000e+08,1.188000e+08,[USD],7084.428576,5567.601319
24,NEAR,NEAR Protocol,https://messari.io/asset/near-protocol,4.260185,7.484358e+08,7.709158e+08,20.393489,2022-01-16T22:00:00Z,2.941864e+00,2022-06-18T19:45:00Z,...,1.523667e+08,12545000.0,0.037500,0.150000,0.082334,1.000000e+09,1.500000e+08,[USD],5074.254239,2024.808389
25,ATOM,Cosmos,https://messari.io/asset/cosmos,10.440489,2.863703e+08,2.946026e+08,44.356138,2022-01-17T00:15:00Z,5.617520e+00,2022-06-18T19:45:00Z,...,2.861840e+07,1629472.0,0.025000,0.080000,0.056938,2.361990e+08,1.889592e+07,[USD],18236.619684,15752.508770
26,XLM,Stellar,https://messari.io/asset/stellar,0.118446,2.516184e+10,4.700000e+10,0.944272,2018-01-04T01:30:00Z,2.720018e-02,2020-03-13T02:00:00Z,...,2.000000e+09,3000000.0,0.001500,0.001500,0.001500,1.000000e+11,1.500000e+08,[None],7796.398832,1892.361940
31,ICP,Internet Computer,https://messari.io/asset/internet-computer,8.641539,2.496826e+08,NaN,491.021188,2021-05-11T03:45:00Z,4.733593e+00,2022-06-18T20:15:00Z,...,3.284496e+07,20540000.0,0.630000,0.630000,0.630000,4.692137e+08,2.956046e+08,[USD],1271.672822,630.377370
35,SAND,The Sandbox,https://messari.io/asset/thesandbox,1.301880,1.284060e+09,1.685220e+09,8.455097,2021-11-25T06:45:00Z,7.426877e-01,2022-06-18T20:30:00Z,...,0.000000e+00,3810000.0,0.005000,0.007200,NaN,5.600000e+08,4.032000e+06,[None],NaN,41457.944934
38,AXS,Axie Infinity,https://messari.io/asset/axie-infinity,17.773372,8.279668e+07,1.468701e+08,164.056661,2021-11-06T19:15:00Z,1.201627e+01,2022-06-18T20:30:00Z,...,2.160000e+06,864000.0,0.080000,0.080000,0.080000,5.998500e+07,4.798800e+06,[USD],22116.715000,30634.652008


In [204]:
df.dtypes

symbol                           object
name                             object
link                             object
current_price                   float64
circ_supply                     float64
liquid_supply                   float64
ath_price                       float64
ath_date                         object
cl_price                        float64
cl_date                          object
mcap                            float64
liquid_mcap                     float64
percent_change_last_1_week      float64
percent_change_last_1_month     float64
percent_change_last_3_months    float64
percent_change_last_1_year      float64
percent_change_month_to_date    float64
percent_change_year_to_date     float64
investors                        object
first_private_sale_date          object
last_private_sale_date           object
num_tokens_private_sale         float64
tot_usd_raised_private_sale     float64
min_price_private_sale          float64
max_price_private_sale          float64
